In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import openpyxl
from openpyxl.styles.borders import Border,Side
import string
from spire.xls import *
from spire.xls.common import *

D:\Users\2346\AppData\Local\Temp\ipykernel_6160\3028338999.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def get_pivot(filename):
    raw_df = pd.read_excel(filename,dtype='object')
    raw_df1 = raw_df[['Employee Code','Employee Name','Division','Department','Section','Designation','Start Date','In Time','Out Time','Work Hours','Status']]
    raw_df1 = raw_df1[(raw_df1['Designation'] != 'Assistant Manager')&(raw_df1['Designation'] != 'Manager')&(raw_df1['Designation'] != 'Senior Manager')&(raw_df1['Designation'] != 'Senior Executive Vice President')&(raw_df1['Designation'] != 'Senior Vice President')&(raw_df1['Designation'] != 'President')&(raw_df1['Designation'] != 'President and CEO')&(raw_df1['Designation'] != 'Vice President')&(raw_df1['Designation'] != 'Deputy Manager')]
    raw_df1['Work Hours'] = raw_df1['Work Hours'].str.replace(':','.').astype('float64')
    pivot_df = raw_df1.pivot(index=['Department','Section','Employee Code','Employee Name'],
                              columns='Start Date',
                              values=['In Time','Out Time','Work Hours','Status'])
    return pivot_df,raw_df1

In [34]:
def get_excel_file(raw_df,df2):
    date_col = raw_df['Start Date'].unique()
    date_col.sort()
    status = ['In Time','Out Time','Work Hours','Status']
    raw_df['Work Minutes'] = raw_df['Work Hours']//1 + raw_df['Work Hours']%1/0.6
    df2 = df2.fillna(method='ffill')
    div = df2['Department'].unique()
    div = np.delete(div , np.where(div == 'Growth Management'))
    section = df2['Section'].unique()
    addition = pd.DataFrame(raw_df['Status'][(raw_df['Status'] == 'Absent')|(raw_df['Status'] == 'Miss Punch Out')].groupby(raw_df['Employee Code']).count()).reset_index()
    addition2 = pd.DataFrame(raw_df['Work Minutes'][raw_df['Work Minutes'] != 0].groupby(raw_df['Employee Code']).mean().round(0)).reset_index()
    test2 = pd.merge(df2,addition,'left',on='Employee Code')
    test2 = pd.merge(test2,addition2,'left',on='Employee Code')
    test2 = test2.rename(columns={'Status_y':'Miss punch','Work Minutes':'Avg work hours'})
    test2[['Miss punch','Avg work hours']] = test2[['Miss punch','Avg work hours']].fillna(0)
    df2 = test2[['Department',
                'Section',
                'Employee Code',
                'Employee Name',
                'Avg work hours',
                'Miss punch',
                'In Time',
                'Out Time',
                'Work Hours',
                'Status_x',
                'In Time.1',
                'Out Time.1',
                'Work Hours.1',
                'Status.1',
                'In Time.2',
                'Out Time.2',
                'Work Hours.2',
                'Status.2',
                'In Time.3',
                'Out Time.3',
                'Work Hours.3',
                'Status.3',
                'In Time.4',
                'Out Time.4',
                'Work Hours.4',
                'Status.4',
                'In Time.5',
                'Out Time.5',
                'Work Hours.5',
                'Status.5',
                'In Time.6',
                'Out Time.6',
                'Work Hours.6',
                'Status.6',
                'In Time.7',
                'Out Time.7',
                'Work Hours.7',
                'Status.7',
                'In Time.8',
                'Out Time.8',
                'Work Hours.8',
                'Status.8',
                'In Time.9',
                'Out Time.9',
                'Work Hours.9',
                'Status.9',
                'In Time.10',
                'Out Time.10',
                'Work Hours.10',
                'Status.10',
                'In Time.11',
                'Out Time.11',
                'Work Hours.11',
                'Status.11',
                'In Time.12',
                'Out Time.12',
                'Work Hours.12',
                'Status.12',
                'In Time.13',
                'Out Time.13',
                'Work Hours.13',
                'Status.13',
                'In Time.14',
                'Out Time.14',
                'Work Hours.14',
                'Status.14',
                'In Time.15',
                'Out Time.15',
                'Work Hours.15',
                'Status.15',
                'In Time.16',
                'Out Time.16',
                'Work Hours.16',
                'Status.16',
                'In Time.17',
                'Out Time.17',
                'Work Hours.17',
                'Status.17',
                'In Time.18',
                'Out Time.18',
                'Work Hours.18',
                'Status.18',
                'In Time.19',
                'Out Time.19',
                'Work Hours.19',
                'Status.19',
                'In Time.20',
                'Out Time.20',
                'Work Hours.20',
                'Status.20',
                'In Time.21',
                'Out Time.21',
                'Work Hours.21',
                'Status.21',
                'In Time.22',
                'Out Time.22',
                'Work Hours.22',
                'Status.22',
                'In Time.23',
                'Out Time.23',
                'Work Hours.23',
                'Status.23',
                'In Time.24',
                'Out Time.24',
                'Work Hours.24',
                'Status.24',
                'In Time.25',
                'Out Time.25',
                'Work Hours.25',
                'Status.25',
                'In Time.26',
                'Out Time.26',
                'Work Hours.26',
                'Status.26',
                'In Time.27',
                'Out Time.27',
                'Work Hours.27',
                'Status.27',
                'In Time.28',
                'Out Time.28',
                'Work Hours.28',
                'Status.28',
                'In Time.29',
                'Out Time.29',
                'Work Hours.29',
                'Status.29']]
    
    ex_div = pd. read_excel('ข้อมูลสายงาน.xlsx')
    vp = [x for x in ex_div['ผู้บริหารสายงาน'].unique().tolist() if type(x) == str]
    vp = vp[6:] ###########################################
    
    for vp in vp:
        dep_list = ex_div['Department_'][ex_div['ผู้บริหารสายงาน'] == vp].tolist()
        for div1 in dep_list:
            df3 = df2[df2['Department'] == div1]
            name = vp + '/' + div1+'_in_out.xlsx'
    
            pd.DataFrame().to_excel(name)
            df3['Miss punch'] = df3['Miss punch'].astype('int32')
            df3 = df3.style.applymap(cond_formatting)
            
            y = 1
            z = 5
            with pd.ExcelWriter(name,engine='openpyxl',mode='a',if_sheet_exists='overlay') as writer :
                df3.to_excel(writer,index=False,startrow=1)
                for i in date_col:
                    for j in status:
                        z = z+1
                        data = {i:['0']}
                        data = pd.DataFrame(data)
                        data1 = {j : ['0']}
                        data1 = pd.DataFrame(data1)
                        data = data[data[i] == '5']
                        data1 = data1[data1[j] == '5']
                        data.to_excel(writer,index=False,startcol=z)
                        data1.to_excel(writer,index=False,startrow=1,startcol=z)
            
            z = 5
            for i in range(date_col.size*4):
                z = z+1
                y = 3
                if z < 26 :
                    start = string.ascii_uppercase[z]
                if z >= 26 :
                    start = string.ascii_uppercase[(z//26)-1] + string.ascii_uppercase[z%26]
                if z+1 < 26 :
                    mid = string.ascii_uppercase[z+1]
                if z+1 >= 26 :
                    mid = string.ascii_uppercase[((z+1)//26)-1] + string.ascii_uppercase[(z+1)%26]
                if z+2 < 26 :
                    mid1 = string.ascii_uppercase[z+2]
                if z+2 >= 26 :
                    mid1 = string.ascii_uppercase[((z+2)//26)-1] + string.ascii_uppercase[(z+2)%26]
                if z+3 < 26 :
                    end1 = string.ascii_uppercase[z+3]
                if z+3 >= 26 :
                    end1 = string.ascii_uppercase[((z+3)//26)-1] + string.ascii_uppercase[(z+3)%26]
    
                if i%4 == 0 :    
                    cell = start + '1' +':' + end1 + '1'
                    wb = openpyxl.load_workbook(name)
                    ws = wb.active
                    ws.merge_cells(cell)
                    for i in range((len(df2[df2['Department'] == div1].index))):
                        cell1 = start + str(y)
                        cell2 = end1 + str(y)
                        cell3 = mid + str(y)
                        cell4 = mid1 + str(y)
                        ws[cell1].border = Border(left = Side(style='thick'))
                        ws[cell2].border = Border(right = Side(style='thick'))
                        ws['A' + str(y)].border = Border(right = Side(style='thin'),left = Side(style='thick'))
                        ws['B' + str(y)].border = Border(right = Side(style='thin'),left = Side(style='thin'))
                        ws['C' + str(y)].border = Border(right = Side(style='thin'),left = Side(style='thin'))
                        ws['D' + str(y)].border = Border(right = Side(style='thick'),left = Side(style='thin'))
                        ws['E' + str(y)].border = Border(right = Side(style='thin'),left = Side(style='thick'))
                        ws['F' + str(y)].border = Border(right = Side(style='thick'),left = Side(style='thin'))
                        ws['A' + '2'].border = Border(right = Side(style='thin'),left = Side(style='thick'),top = Side(style='thick'),bottom = Side(style='thick'))
                        ws['B' + '2'].border = Border(right = Side(style='thin'),left = Side(style='thin'),top = Side(style='thick'),bottom = Side(style='thick'))
                        ws['C' + '2'].border = Border(right = Side(style='thin'),left = Side(style='thin'),top = Side(style='thick'),bottom = Side(style='thick'))
                        ws['D' + '2'].border = Border(right = Side(style='thick'),left = Side(style='thin'),top = Side(style='thick'),bottom = Side(style='thick'))
                        ws['E' + '2'].border = Border(right = Side(style='thin'),left = Side(style='thick'),top = Side(style='thick'),bottom = Side(style='thick'))
                        ws['F' + '2'].border = Border(right = Side(style='thick'),left = Side(style='thin'),top = Side(style='thick'),bottom = Side(style='thick'))
                        ws[start + '2'].border = Border(right = Side(style='thin'),left = Side(style='thick'),top = Side(style='thick'),bottom = Side(style='thick'))
                        ws[mid + '2'].border = Border(right = Side(style='thin'),left = Side(style='thin'),top = Side(style='thick'),bottom = Side(style='thick'))
                        ws[mid1 + '2'].border = Border(right = Side(style='thin'),left = Side(style='thin'),top = Side(style='thick'),bottom = Side(style='thick'))
                        ws[end1 + '2'].border = Border(right = Side(style='thick'),left = Side(style='thin'),top = Side(style='thick'),bottom = Side(style='thick'))
                        ws[start + '1'].border = Border(right = Side(style='thin'),left = Side(style='thick'))
                        ws[end1 + '1'].border = Border(right = Side(style='thin'),left = Side(style='thick'))
                        
                        if y == (len(df2[df2['Department'] == div1].index)+2):
                            ws['A' + str(y)].border = Border(right = Side(style='thin'),left = Side(style='thick'),bottom = Side(style='thick'))
                            ws['B' + str(y)].border = Border(right = Side(style='thin'),left = Side(style='thin'),bottom = Side(style='thick'))
                            ws['C' + str(y)].border = Border(right = Side(style='thin'),left = Side(style='thin'),bottom = Side(style='thick'))
                            ws['D' + str(y)].border = Border(right = Side(style='thin'),left = Side(style='thin'),bottom = Side(style='thick'))
                            ws['E' + str(y)].border = Border(right = Side(style='thin'),left = Side(style='thick'),bottom = Side(style='thick'))
                            ws['F' + str(y)].border = Border(right = Side(style='thin'),left = Side(style='thin'),bottom = Side(style='thick'))
                            ws[cell1].border = Border(bottom = Side(style='thick'),left = Side(style='thick'))
                            ws[cell2].border = Border(bottom = Side(style='thick'),right = Side(style='thick'))
                            ws[cell3].border = Border(bottom = Side(style='thick'))
                            ws[cell4].border = Border(bottom = Side(style='thick'))
                        y = y+1
                    wb.save(name)
    


In [ ]:
def get_data(filename):
    
    raw_df = pd.read_excel('Master_Attendance_Report_20240206_114132_023888.xlsx')
    ex_div = pd. read_excel('ข้อมูลสายงาน.xlsx')
    
    raw_df1 = raw_df[(raw_df['Designation'] == 'Analyst')|(raw_df['Designation'] == 'Associate')]
    raw_df1 = raw_df1[['Employee Code','Employee Name','Division','Department','Section','Designation','Start Date','In Time','Out Time','Work Hours','Status']]
    raw_df1['Work Hours'] = raw_df1['Work Hours'].str.replace(':','.').astype('float64')
    
    miss_punch = raw_df1[['Employee Code','Status']][(raw_df1['Status']== 'Absent')|(raw_df1['Status']=='Miss Punch Out')].groupby('Employee Code').count().reset_index()
    miss_punch = miss_punch.rename(columns={'Status':'Miss Punch'})
    miss_punch_final = miss_punch.merge(raw_df1.drop(columns=['Start Date','In Time','Out Time','Work Hours','Status','Designation']),'inner',on='Employee Code').drop_duplicates()
    miss_punch_final = miss_punch_final[miss_punch_final['Miss Punch'] > 6]
    miss_punch_final = miss_punch_final.merge(ex_div,'left',left_on='Department',right_on='Department_').drop(columns=['Division_','Department_'])

    df_work_hours = raw_df1[['Employee Code','Work Hours']][raw_df1['Status'] == 'Present']
    df_work_hours['Work Minutes'] = (df_work_hours['Work Hours']//1*60)+(df_work_hours['Work Hours']%1*100)
    df_work_hours = df_work_hours[['Employee Code','Work Minutes']].groupby('Employee Code').mean().reset_index()
    df_work_hours['Avg Work Hours'] = ((df_work_hours['Work Minutes']/60)//1) + (((df_work_hours['Work Minutes']/60)%1)*0.6).round(2)
    
    work_hours = df_work_hours[['Employee Code','Avg Work Hours']]
    miss_punch_final = miss_punch_final.merge(work_hours,'left',on='Employee Code')
    miss_punch_final['Avg Work Hours'] = miss_punch_final['Avg Work Hours'].fillna(0)
    work_hours = work_hours.merge(raw_df1.drop(columns=['Start Date','In Time','Out Time','Work Hours','Status','Designation']),'inner',on='Employee Code').drop_duplicates()
    work_hours = work_hours.merge(miss_punch,'left',on='Employee Code')
    work_hours['Miss Punch'] = work_hours['Miss Punch'].fillna(0)
    work_hours['Miss Punch'] = work_hours['Miss Punch'].astype(int)
    work_hours_final = work_hours[work_hours['Miss Punch'] < 7]
    work_hours_final = work_hours_final.merge(ex_div,'left',left_on='Department',right_on='Department_').drop(columns=['Division_','Department_'])
    
    return work_hours_final,miss_punch_final,ex_div

In [ ]:
def get_file(ex_div):
    
    vp = [x for x in ex_div['ผู้บริหารสายงาน'].unique().tolist() if type(x) == str]
    
    for i in vp:
        x = 0
        dep_list = ex_div['Department_'][ex_div['ผู้บริหารสายงาน'] == i].tolist()
        for j in dep_list:
            work_hour = work_hours_final[work_hours_final['Department'] == j].sort_values('Avg Work Hours',ascending=False).head(3)
            work_hour = work_hour[['Department','Section','Employee Code','Employee Name','Avg Work Hours','Miss Punch']].reset_index().drop(columns='index')
            miss_punch = miss_punch_final[miss_punch_final['Department'] == j].sort_values('Miss Punch',ascending=False).head(3)
            miss_punch = miss_punch[['Department','Section','Employee Code','Employee Name','Avg Work Hours','Miss Punch']].reset_index().drop(columns='index')
    
            with pd.ExcelWriter(i+'/attendance summary.xlsx',engine='openpyxl',mode='a',if_sheet_exists='overlay') as writer :
                work_hour = work_hour.style.applymap(cond_formatting)
                work_hour.to_excel(writer,index=False,startrow=x,sheet_name='Sheet1')
                miss_punch = miss_punch.style.applymap(cond_formatting)
                miss_punch.to_excel(writer,index=False,startrow=(x+4),sheet_name='Sheet1',header=False)
        
    
            print(j)
            for k in range(7):
                wb = openpyxl.load_workbook(i+'/attendance summary.xlsx')
                ws = wb.active
                if k == 0:
                    ws['A'+ str(x+k+1)].border = Border(left = Side(style='thick'),right = Side(style='thin'),top = Side(style='thick'),bottom = Side(style='thick'))
                    ws['B'+ str(x+k+1)].border = Border(left = Side(style='thin'),right = Side(style='thin'),top = Side(style='thick'),bottom = Side(style='thick'))
                    ws['C'+ str(x+k+1)].border = Border(left = Side(style='thin'),right = Side(style='thin'),top = Side(style='thick'),bottom = Side(style='thick'))
                    ws['D'+ str(x+k+1)].border = Border(left = Side(style='thin'),right = Side(style='thick'),top = Side(style='thick'),bottom = Side(style='thick'))
                    ws['E'+ str(x+k+1)].border = Border(left = Side(style='thick'),right = Side(style='thin'),top = Side(style='thick'),bottom = Side(style='thick'))
                    ws['F'+ str(x+k+1)].border = Border(left = Side(style='thin'),right = Side(style='thick'),top = Side(style='thick'),bottom = Side(style='thick'))
                elif (k == 3) | (k == 6):
                    ws['A'+ str(x+k+1)].border = Border(left = Side(style='thick'),bottom = Side(style='thick'))
                    ws['B'+ str(x+k+1)].border = Border(bottom = Side(style='thick'))
                    ws['C'+ str(x+k+1)].border = Border(bottom = Side(style='thick'))
                    ws['D'+ str(x+k+1)].border = Border(right = Side(style='thick'),bottom = Side(style='thick'))
                    ws['E'+ str(x+k+1)].border = Border(left = Side(style='thick'),bottom = Side(style='thick'))
                    ws['F'+ str(x+k+1)].border = Border(right = Side(style='thick'),bottom = Side(style='thick'))
                else:
                    ws['A'+ str(x+k+1)].border = Border(left = Side(style='thick'))
                    ws['D'+ str(x+k+1)].border = Border(right = Side(style='thick'))
                    ws['F'+ str(x+k+1)].border = Border(right = Side(style='thick'))
                
                wb.save(i+'/attendance summary.xlsx')
            
            x+=12

In [4]:
def cond_formatting(x):
    if type(x) == float:
        if x == 0:
            return 'background-color: yellow; color: red; font-weight: bold'
        elif x < 7.45:
            return 'background-color: lightgrey; color: red; font-weight: bold'
    if type(x) == int:
        if x > 11 :
            return 'background-color: lightgrey; color: red; font-weight: bold'
    else:
        return None

In [5]:
df,raw_df = get_pivot('Master_Attendance_Report_20240206_114132_023888.xlsx')

In [6]:
raw_df

,Employee Code,Employee Name,Division,Department,Section,Designation,Start Date,In Time,Out Time,Work Hours,Status
31,0157,WICHITRA RATANARUAMKARN,Finance and Investment,Finance,Finance,Associate,2024-01-01,0:00,0:00,0.00,Holiday
32,0157,WICHITRA RATANARUAMKARN,Finance and Investment,Finance,Finance,Associate,2024-01-13,0:00,0:00,0.00,Weekly Off
33,0157,WICHITRA RATANARUAMKARN,Finance and Investment,Finance,Finance,Associate,2024-01-14,0:00,0:00,0.00,Weekly Off
34,0157,WICHITRA RATANARUAMKARN,Finance and Investment,Finance,Finance,Associate,2024-01-15,7:32,18:06,9.34,Present
35,0157,WICHITRA RATANARUAMKARN,Finance and Investment,Finance,Finance,Associate,2024-01-16,6:52,18:47,10.55,Present
...,...,...,...,...,...,...,...,...,...,...,...
25506,dummy7,Dummy Seven,Corporate Administration,Part Time,Accident Registration,Analyst,2024-01-16,0:00,0:00,0.00,Absent
25507,dummy7,Dummy Seven,Corporate Administration,Part Time,Accident Registration,Analyst,2024-01-17,0:00,0:00,0.00,Absent
25508,dummy7,Dummy Seven,Corporate Administration,Part Time,Accident Registration,Analyst,2024-01-27,0:00,0:00,0.00,Weekly Off
25509,dummy7,Dummy Seven,Corporate Administration,Part Time,Accident Registration,Analyst,2024-01-28,0:00,0:00,0.00,Weekly Off


In [7]:
#df.to_excel('in_out_all.xlsx')

# เปลี่ยน shape ไฟล์รวมก่อน

In [6]:
df2 = pd.read_excel('in_out_all.xlsx',skiprows=[0])

In [39]:
get_excel_file(raw_df,df2)

D:\Users\2346\AppData\Local\Temp\ipykernel_6160\3817387153.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df2 = df2.fillna(method='ffill')
D:\Users\2346\AppData\Local\Temp\ipykernel_6160\3817387153.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Miss punch'] = df3['Miss punch'].astype('int32')
D:\Users\2346\AppData\Local\Temp\ipykernel_6160\3817387153.py:155: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df3 = df3.style.applymap(cond_formatting)
D:\Users\2346\AppData\Local\Temp\ipykernel_6160\3817387153.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [ ]:
work_hours_final,miss_punch_final,ex_div = get_data('Master_Attendance_Report_20240206_114132_023888.xlsx')

In [ ]:
get_file(ex_div)